## Environment 
Packages:

In [1]:
pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


LangSmith:

In [2]:
import os
%pip install python-dotenv
from dotenv import load_dotenv

# Load the environment variables from .env file
load_dotenv()

# Access your API key
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TICKETMASTER_API_KEY = os.getenv("TICKETMASTER_API_KEY")

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Load the Relative event data

In [3]:
mongo_jarname = 'mongo-spark-connector_2.13-10.1.1-all.jar'
mongo_package_name = 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1'

In [4]:
# download the spark mongo connector
!curl -o mongo-spark-connector_2.13-10.1.1-all.jar "https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.13/10.1.1/mongo-spark-connector_2.13-10.1.1-all.jar"  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2323k  100 2323k    0     0  44.4M      0 --:--:-- --:--:-- --:--:-- 44.4M


In [5]:
import os

current_dir = os.getcwd()
print(current_dir)
jar_location = current_dir + "/"+mongo_jarname
jar_location

/home/jovyan/RAG_DE


'/home/jovyan/RAG_DE/mongo-spark-connector_2.13-10.1.1-all.jar'

In [6]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .config("spark.jars.packages", mongo_package_name)\
    .appName("Enhanced Data Warehouse ETL") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-46c12460-b331-4972-a2d7-24eddb68fe2c;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 481ms :: artifacts dl 74ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts

In [7]:
import requests
import json
from pymongo import MongoClient

def retrieve_and_save_events_json():
    # Ticketmaster API call
    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    params = {
        "keyword": 'Enhypen',
        "apikey": TICKETMASTER_API_KEY 
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        # Save JSON data to a file
        with open('events.json', 'w') as f:
            f.write(response.text)

        print("Events data saved to events.json.")
    else:
        print("Error:", response.status_code)
        
# First, retrieve and save the events JSON
retrieve_and_save_events_json()

Events data saved to events.json.


In [8]:
import json
from pymongo import MongoClient
import pyspark.pandas as ps

events_df = spark.read.json("events.json")

events_df.write.format("mongo") \
    .mode("overwrite") \
    .option("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/ticketmaster_data.events") \
    .save()

/usr/local/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [9]:
# Read data from MongoDB into a DataFrame
events_df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("spark.mongodb.input.uri","mongodb://127.0.0.1:27017/ticketmaster_data.events").load()

# Show the data 
events_df.show()

# Show the schema
events_df.printSchema()

+--------------------+--------------------+--------------------+-------------+
|           _embedded|                 _id|              _links|         page|
+--------------------+--------------------+--------------------+-------------+
|{[{{[{{{/discover...|{663005a3414f2c16...|{{/discovery/v2/e...|{0, 20, 5, 1}|
+--------------------+--------------------+--------------------+-------------+

root
 |-- _embedded: struct (nullable = true)
 |    |-- events: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _embedded: struct (nullable = true)
 |    |    |    |    |-- attractions: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- _links: struct (nullable = true)
 |    |    |    |    |    |    |    |-- self: struct (nullable = true)
 |    |    |    |    |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |    |    |    |-- classifications: array (nul

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Simplify Nested Schema") \
    .getOrCreate()

# Selecting the _embedded.events field and exploding it
df_events = events_df.select(explode(col("_embedded.events")).alias("event"))

# Exploding attractions and selecting relevant fields
df_attractions = df_events.select(
    col("event.id").alias("event_id"),
    col("event.name").alias("event_name"),
    col("event.url").alias("event_url"),
    explode(col("event._embedded.attractions")).alias("attraction")
).select(
    "event_id",
    "event_name",
    "event_url",
    col("attraction.id").alias("attraction_id"),
    col("attraction.name").alias("attraction_name"),
    col("attraction.classifications.genre.name").alias("attraction_genre")
)

# Exploding venues and selecting relevant fields
df_venues = df_events.select(
    col("event.id").alias("event_id"),
    explode(col("event._embedded.venues")).alias("venue")
).select(
    "event_id",
    col("venue.name").alias("venue_name"),
    col("venue.city.name").alias("venue_city"),
    col("venue.country.name").alias("venue_country")
)

# Joining attractions and venues on event_id
df_combined = df_attractions.join(df_venues, "event_id", "inner")

# Dropping rows with any null values in the columns we are focusing on
events_cleaned = df_combined.dropna()

# Show the result
events_cleaned.show(truncate=False)


24/04/29 21:40:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------------+--------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------+---------------+----------------+--------------+--------------------------+------------------------+
|event_id        |event_name                            |event_url                                                                                                                       |attraction_id|attraction_name|attraction_genre|venue_name    |venue_city                |venue_country           |
+----------------+--------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------+---------------+----------------+--------------+--------------------------+------------------------+
|1AYZkvvGkeUc0gD |ENHYPEN World Tour 'Fate Plus' In U.S.|https://www.ticketmaster.com/enhypen-wor

In [11]:
events_cleaned.write.format("mongo").mode("append").option("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/ticketmaster_data.events").save()

In [12]:
# Create Parquet file
events_cleaned.write.mode("overwrite").parquet("events_cleaned.parquet")

## 2. Load Wekipidia data

In [13]:
# Import packages
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Enhypen'
r = requests.get(url)

# Extract the response as html
html_doc = r.text

# Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html_doc)

# Get all text from the page
enhypen_text = soup.get_text()

# Assuming 'enhypen_text' is your variable containing the text
index_references = enhypen_text.find("References")

# Find the index of the first occurrence of "References"
if index_references != -1:
    # Find the index of the second occurrence of "References"
    second_index_references = enhypen_text.find("References", index_references + 1)
    if second_index_references != -1:
        # Extract text up to the second occurrence of "References"
        extracted_text = enhypen_text[:second_index_references]
    else:
        # If only one occurrence of "References" exists, extract text until the end
        extracted_text = enhypen_text
else:
    # If "References" doesn't exist, take the entire text
    extracted_text = enhypen_text
    
extracted_text

print(extracted_text)






Enhypen - Wikipedia



































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search





























Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1Name







2Career



Toggle Career subsection





2.1Pre-debut activities and formation through I-Land







2.22020–2021: debut with Border: Day One, second extended play, first Japanese single and Dimension: Dilemma







2.32022: Dimension: Answer, second Japanese single, Manifesto: Day 1 and first Japanese studio album







2.42023–present: Dark Blood, third Japanese single, Orange Blood and

In [14]:
import re

# Assuming 'enhypen_text' is your variable containing the text
enhypen_text_cleaned = re.sub(r'\[.*?\]|\n\s*\n', '\n', extracted_text)

print(enhypen_text_cleaned)


Enhypen - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
		Navigation
Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate
		Contribute
HelpLearn to editCommunity portalRecent changesUpload file
Search
Search
Create account
Log in
Personal tools
 Create account Log in
		Pages for logged out editors learn more
ContributionsTalk
Contents
move to sidebar
hide
(Top)
1Name
2Career
Toggle Career subsection
2.1Pre-debut activities and formation through I-Land
2.22020–2021: debut with Border: Day One, second extended play, first Japanese single and Dimension: Dilemma
2.32022: Dimension: Answer, second Japanese single, Manifesto: Day 1 and first Japanese studio album
2.42023–present: Dark Blood, third Japanese single, Orange Blood and special album
3Members
4Endorsements
5Discography
6Filmography
Toggle Filmography subsection
6.1Film
6.2Television
6.3Online shows
6.4Radio
7Concerts and tours
8Awards and nominations
9References
10External links
Toggle

In [15]:
# Save the cleaned text to a .txt file
with open('enhypen_text_cleaned.txt', 'w') as file:
    file.write(enhypen_text_cleaned)

# Read the cleaned text from the file
with open('enhypen_text_cleaned.txt', 'r') as file:
    enhypen_text_cleaned = file.read()


In [16]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("LoadTextToMongoDB") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/enhypen.info") \
    .getOrCreate()

# Create DataFrame with the cleaned text
cleaned_text_df = spark.createDataFrame([(enhypen_text_cleaned,)], ["text"])

# Write the DataFrame to MongoDB
cleaned_text_df.write.format("mongo").mode("overwrite").option("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/enhypen.info").save()

24/04/29 21:40:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [17]:
# Read back from MongoDB into a DataFrame

cleaned_enhypen_text = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("spark.mongodb.input.uri","mongodb://127.0.0.1:27017/enhypen.info").load()
cleaned_enhypen_text.show()

+--------------------+--------------------+
|                 _id|                text|
+--------------------+--------------------+
|{663005b7414f2c16...|\nEnhypen - Wikip...|
+--------------------+--------------------+



In [18]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# List all databases
print("Databases:")
print(client.list_database_names())

# Iterate over databases and list collections
for db_name in client.list_database_names():
    db = client[db_name]
    print(f"\nCollections in database '{db_name}':")
    print(db.list_collection_names())

Databases:
['admin', 'config', 'enhypen', 'local', 'ticketmaster_data']

Collections in database 'admin':
['system.version']

Collections in database 'config':
['system.sessions']

Collections in database 'enhypen':
['info']

Collections in database 'local':
['startup_log']

Collections in database 'ticketmaster_data':
['events']


In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("CombineContent") \
    .getOrCreate()

# Read the Parquet file into a DataFrame
parquet_df = spark.read.parquet("events_cleaned.parquet")

# Read the text file into a DataFrame
text_df = spark.read.text("enhypen_text_cleaned.txt")

# Convert each row of the DataFrame into a list of strings
text_content_list = text_df.rdd.map(lambda x: x[0]).collect()

# Concatenate the strings with newline characters
text_content = '\n'.join(text_content_list)

# Select relevant columns and concatenate their values
combined_content_df = parquet_df.select(
    concat_ws("event_id", "event_name", "event_url", "attraction_id", "attraction_name", "attraction_genre", "venue_name", "venue_city", "venue_country").alias("combined_content")
)

# Extract the combined content from Parquet DataFrame
combined_content_list = combined_content_df.rdd.map(lambda x: x[0]).collect()

# Concatenate the content from both DataFrames
combined_content = '\n'.join(combined_content_list + [text_content])

# Write the combined content to a new text file
with open("combined_content.txt", "w") as file:
    file.write(combined_content)


24/04/29 21:40:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## 3. Indexing

In [20]:
# Load the document
question_1 = "Give me some information about'Fate Plus'."
document = combined_content

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents([document])

# Make splits
splits = text_splitter.split_text(document)

## Retrieval

In [22]:
# Index
__import__('pysqlite3')
import pysqlite3
import sys
sys.modules['sqlite3'] = sys.modules["pysqlite3"]

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.openai import OpenAIEmbeddings

# Initialize embeddings with a specific OpenAI model
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

vectorstore = Chroma.from_texts(texts=splits, embedding=embedding)

# Create a retriever for searching within the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

/usr/local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Generation

In [23]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [24]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Chain
chain = prompt | llm

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke(question_1)

"'Fate Plus' is the name of ENHYPEN's world tour."

In [26]:
question_2 = "who is ENHYPEN? Give me some information."
rag_chain.invoke(question_2)

'ENHYPEN is a South Korean boy group formed through the reality survival show "I-LAND." They are stylized in all caps and their name is written as 엔하이픈 in Korean, Enhaipeun in RR, and エンハイプン in Japanese.'